## Initial setup so we have something

In [ ]:
from pulp import *
import random
problem = LpProblem("Exam_Scheduling_Problem", LpMinimize)

#these sets need to be populated with actual data
E = ["E1", "E2", "E3"]  # exams
S = ["S1", "S2", "S3", "S4", "S5"]  # students
V = ["V1", "V2"]  # venues

D = range(1, 16)      # days 1–15
J= range(1, 4)       # 3 sessions per day

Timeslots = [(d, j) for d in D for j in J]


# cost[(e,v,t)]
cost = {}  

for e in E:
    for v in V:
        for t in Timeslots:
            cost[(e, v, t)] = random.randint(1, 10)

# bs[(s,e)] = 1 if student s registered for exam e, else 0
bs = {}

enrollments = {
    "S1": ["E1"],
    "S2": ["E1", "E2"],
    "S3": ["E2"],
    "S4": ["E3"],
    "S5": ["E1", "E3"]
}

for s in S:
    for e in E:
        bs[(s, e)] = 1 if e in enrollments.get(s, []) else 0

# Ne[e] = number of students in exam e
Ne = {
    "E1": 3,
    "E2": 2,
    "E3": 2
}

# cV[v] = capacity of venue v
cV = {
    "V1": 5,
    "V2": 3
}

x = pulp.LpVariable.dicts(
    "x",
    ((e, v, t) for e in E for v in V for t in Timeslots),
    cat="Binary"
)

#Objective Function
problem += lpSum(cost[e, v, t] * x[e, v, t] for e in E for v in V for t in Timeslots)

#Constraint 1: Each exam must be assigned to exactly one venue and time
for e in E:
    problem+= lpSum(x[e, v, t] for v in V for t in Timeslots) == 1, f"Exam_{e}_scheduled_once"

#Constraint 2: No student can have overlapping exams
for s in S: 
    for t in Timeslots:
        problem += lpSum(bs[s, e] * x[e, v, t] for e in E for v in V) <= 1, f"Student_{s}_no_overlap_at_time_{t}"

#Constraint 3: Venue capacity must not be exceeded
for v in V:
    for t in Timeslots:
        problem += lpSum(x[e, v, t] for e in E) <= cV[v], f"Venue_{v}_capacity_at_time_{t}"

problem.solve()

print("Status:", LpStatus[problem.status])

for e in E:
    for v in V:
        for t in Timeslots:
            if x[e, v, t].value() == 1:
                print(f"{e} scheduled in {v} at time {t}")
